# A. Einleitung (Projektziele, Vorgehen, Schwerpunkte)
Im Rahmen des Abschlussprojekts zum Kurs "Data Analytics" wollen wir eine ausführliche explorative Datenanalyse (EDA) der Daten zu Energieverbräuchen und Bruttoinlandsprodukt ausgewählter europäischer Länder auf Quartals- und Jahresbasis vornehmen. 

Dafür bedienen wir uns der Daten aus einer zentralen pickle-Datei. Diese wiederum wurden in einem separaten Jupyter-Notebook erstellt. In diesem wiederum wurden die ursprünglichen Quelldaten von eurostat (https://ec.europa.eu/eurostat/en/) geladen, aufbereitet und deskriptiv analysiert.

Der Schwerpunkt der EDA dieses Notebooks soll dabei auf der Visualisierung von Größenverhältnissen und zeitlichen Verläufen, sowie saisonalen und geografischen Trends liegen. Dabei werden hauptsächlich Plots und Dashboards aus den Python-Bibliotheken Plotly Express / Dash verwendet.

Qualitativ wollen wir vor allem die sogenannte <b>"absolute Entkopplung"</b> des Energieverbrauchs vom Bruttoinlandsprodukt beleuchten.
- Dieser volkswirtschaftliche Begriff bezeichnet ein Verhalten über die Zeit, bei dem das Bruttoinlandsprodukt gleich bleibt oder steigt, während der Energieverbrauch fällt.
- Wir wollen dabei eine differenziertere Betrachtung zwischen <b>kurzfristiger Entkopplung</b> und <b>langfristiger Entkopplung</b> vornehmen. Diese Begriffe werden wir an geeigneter Stelle erklären.

Des weiteren wollen wir die sogenannte <b>"Energieproduktivität"</b> betrachten (die indirekt mit der absoluten Entkopplung zusammenhängt, wie wir sehen werden).
- Diese volkswirtschaftliche Größe ergibt sich aus dem Quotienten von Bruttoinlandsprodukt und Energieverbrauch. Sie ist ein Indikator für effizienten Energieeinsatz eines Landes (hohes BIP bei geringem Energieverbrauch ergibt eine hohe Energieproduktivität).

Dafür wollen wir uns an Leitfragen orientieren, die in einem der nächsten Kapiteln zu Beginn formuliert werden. In jedem Teilkapitel werden wir die gewonnenen Erkenntnisse resümieren.

Wir werden dabei die Daten auf verschiedenen Ebenen durchleuchten und "von oben nach unten" hierarchisch durch die Daten gehen. Das heißt, wir beginnen "oben" auf gesamteuropäischer Ebene und arbeiten uns dann ins einzelne Land "nach unten". Danach werden wir noch geografische Visualisierungen vornehmen und auf saisonale Trends untersuchen.

# B. Importe und Styling-Parameter
Wir importieren im Folgenden die benötigten Bibliotheken, die pickle-Datei, die uns die nötigen Dataframes liefert, und Zusatzwerte für das Styling der Plots und Dashboards

In [1]:
import pickle
import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import plotly.io as pio
import sys
import os
# from global_styling_parameters import background_color, html_elements_color, global_border_radius, gdp_color, energy_color, prod_color

# Absoluten Pfad des Projektordners ermitteln
projektordner = os.getcwd()
# Zum sys.path hinzufügen, falls noch nicht enthalten
if projektordner not in sys.path:
    sys.path.append(projektordner)

#################### DataFrames laden ################
with open(r"cleaned_dataframes.pkl", "rb") as file:
    df_loaded = pickle.load(file)

df_Y = df_loaded["df_Y"]
df_Q = df_loaded["df_Q"]

###################### adjust global template ###############
###################### Styling Parameters ###############
background_color = "white"
html_elements_color = "#001124"
global_border_radius = "5px"
gdp_color = "#0057c4"
energy_color = "#b50000"
prod_color = "#008200"
pio.templates["custom_template"] = pio.templates["plotly_dark"]
pio.templates["custom_template"].layout.update({
'paper_bgcolor': html_elements_color,
'plot_bgcolor': html_elements_color,
})


Layout({
    'annotationdefaults': {'arrowcolor': '#f2f5fa', 'arrowhead': 0, 'arrowwidth': 1},
    'autotypenumbers': 'strict',
    'coloraxis': {'colorbar': {'outlinewidth': 0, 'ticks': ''}},
    'colorscale': {'diverging': [[0, '#8e0152'], [0.1, '#c51b7d'], [0.2,
                                 '#de77ae'], [0.3, '#f1b6da'], [0.4, '#fde0ef'],
                                 [0.5, '#f7f7f7'], [0.6, '#e6f5d0'], [0.7,
                                 '#b8e186'], [0.8, '#7fbc41'], [0.9, '#4d9221'],
                                 [1, '#276419']],
                   'sequential': [[0.0, '#0d0887'], [0.1111111111111111,
                                  '#46039f'], [0.2222222222222222, '#7201a8'],
                                  [0.3333333333333333, '#9c179e'],
                                  [0.4444444444444444, '#bd3786'],
                                  [0.5555555555555556, '#d8576b'],
                                  [0.6666666666666666, '#ed7953'],
                           

Wir haben hiermit insbesondere die beiden zentralen Datensätze df_Y und df_Q importiert. Sie enthalten die Länderwerte für BIP und Energieverbrauch einmal auf Quartalsebene (df_Q) und auf Jahresebene (df_Y).

# 0. Leitfragen

Um mit unserer explorativen Datenanalyse beginnen zu können, formulieren wir uns zunächst folgende Leitfragen:

Dabei wollen wir fortlaufend die Stück für Stück gewonnenen Erkenntnisse zu folgenden Leitfragen sammeln:
<div style="border: 2px solid black; padding: 10px; background-color: white;">
    <ul style="list-style-type: circle;">
            <li>Verteilung:
                <ul style="list-style-type: circle;">
                    <li>Gibt es Länder, die bezüglich einer (oder mehrerer) der Größen BIP, Energieverbrauch, Energieproduktivität dominieren?</li>
                    <li>Wie verhält es sich bei diesen Ländern in den jeweils anderen Größen?</li>
                </ul>
            </li>
            <li>Absolute Entkopplung:
                <ul style="list-style-type: circle;">
                    <li>Gibt es einen Zusammenhang zwischen hohem BIP und hohem Energieverbrauch?</li>
                    <li>Gibt es einen Trend davon weg?</li>
                    <li>Welchen Ländern gelingt es, sich zu entkoppeln (BIP steigt, Energieverbrauch sinkt)?</li>
                </ul>
            </li>
            <li> Energieproduktivität:
                <ul style="list-style-type: circle;">
                    <li>Gibt es bei der Energieproduktivität einen Zusammenhang zur Wirtschaftsstärke (BIP) oder der absoluten Entkopplung (AE) eines Landes?
                    </li>
                </ul>
            </li>
            <li> Saisonalität:
                <ul style="list-style-type: circle;">
                    <li>Sind saisonale Trends bei BIP, Energieverbrauch und Energieproduktivität erkennbar?
                    </li>
                </ul>
            </li>
            <li> Zusatzerkenntnisse:
                <ul style="list-style-type: circle;">
                    <li>Welche weiteren interessanten Beobachtungen können wir machen?
                    </li>
                </ul>
            </li>
        </ul>
</div>

Wir werden in den folgenden Kapiteln Stück für Stück anhand der Visualisierungen Erkenntnisse zu diesen Leitfragen(gruppen) gewinnen. Am Ende jedes Kapitels werden wir diese unter den übergeordneten Leitfragengruppen zusammenfassen. Wenn eine Leitfragengruppe in einer Zusammenfassung weggelassen wird, bedeutet das, dass für diese Grupe im Kapitel keine oder keine relevanten Erkenntnisse gewonnen wurden. Auf die Wiederaufführung der Unterfragen verzichten wir dort aus Platzgründen. Nun beginnen wir mit der EDA.

<b style="color:red;">Hinweis: Wir werden im Verlauf des Notebooks die Abkürzungen "BIP" (= Bruttoinlandsprodukt), "EV" (= Energieverbrauch), "EP" (= Energieproduktivität) und "AE" (absolute Entkopplung) verwenden!</b>

# 1. Kumulierte Betrachtung, Korrelation BIP/Energieverbrauch, europäische Durchschnittswerte
Wir wollen beginnen, indem wir Europa als Gesamtheit betrachten: Wir ziehen kumulierte Werte und europäische Durchschnitte heran. 

Dabei verwenden wir Dashboards und Plots. Nach jedem Teilkapitel mit Visualisierung resümieren wir die Erkenntnisse, die wir in Bezug auf unsere Leitfragen aus der Visualisierung entnehmen konnten.

## 1.1 Kumulierte Betrachtung

### 1.1.1 Feature Engineering der benötigten Spalten
Zur Untersuchung obiger Fragestellungen führen wir zunächst einige Spalten ein. Wir legen auf Quartals- und Jahresebene folgende Spalten an:
- Energieproduktivität (Mio€/GWh): Quotient aus BIP und Energieverbrauch
- Kumulatives BIP (je Land): fortlaufende Summe der Quartalswerte bzw. Jahreswerte des BIP
- Kumulativer Energieverbrauch (je Land): fortlaufende Summe der Quartalswerte bzw. Jahreswerte des Energieverbrauchs

In [2]:
# Berechnung Energieproduktivität (BIP / Energieverbrauch) und kumulative Summen...
# ...einmal für den Dataframe mit den Jahreswerten....
df_Y["Energieproduktivität (Mio€/GWh)"] = df_Y["BIP (Mio €)"] / df_Y["Energieverbrauch (GWh)"]
df_Y["Energieproduktivität (Mio€/GWh)"] = df_Y["Energieproduktivität (Mio€/GWh)"].round(2)
df_Y["Kumulativer Energieverbrauch (GWh)"] = df_Y.groupby("Land")["Energieverbrauch (GWh)"].cumsum()
df_Y["Kumulatives BIP (Mio €)"] = df_Y.groupby("Land")["BIP (Mio €)"].cumsum()
# ...und einmal für den Dataframe mit den Quartalswerten....
df_Q["Energieproduktivität (Mio€/GWh)"] = df_Q["BIP (Mio €)"] / df_Q["Energieverbrauch (GWh)"]
df_Q["Energieproduktivität (Mio€/GWh)"] = df_Q["Energieproduktivität (Mio€/GWh)"].round(2)
df_Q["Kumulativer Energieverbrauch (GWh)"] = df_Q.groupby("Land")["Energieverbrauch (GWh)"].cumsum()
df_Q["Kumulatives BIP (Mio €)"] = df_Q.groupby("Land")["BIP (Mio €)"].cumsum()

### 1.1.2 Visualisierung

In [3]:
from dashboards_and_tables.D1_cumulations.app import run_cumulations_app

run_cumulations_app(df_Y, df_Q)

### 1.1.3 Beobachtungen und Haupterkenntnisse

<b>Verteilung</b>:
- Wir können in den gestapelten Liniendiagrammen mit akribischem Blick bereits grob erkennen, dass Deutschland, Frankreich, Italien, Spanien, Niederlande und Türkei bei BIP und EV dominieren. Das werden wir später (insbesondere bei den Rankings) aber noch genauer sehen. Wir kürzen in der Folge diese Ländergruppe mit der Anfangsbuchstabenkombination "DFISNT" ab.
- Bei DFISNT sehen, wir dass hohes BIP auch hoher EV bedeutet. Insbesondere die Türkei hat einen enorm hohen EV. Über die EP sehen wir noch nichts.
- Aufgrund von DFISNT und der Ländersummen im oberen Plot lässt sich also ein Zusammenhang zwischen hohem BIP und hohem EV vermuten. Wir werden jedoch noch sehen, dass es auch entgegengesetzte Trends gibt.

<b>Absolute Entkopplung</b>:
- Wir können in den (nicht-kumulierten) Jahreswerten im Plot der Ländersummen (oberer Graph im Dashboard) bereits erkennen, dass das europäische jährliche BIP steigt, während der europäische EV langsam sinkt. 
- Damit sehen wir direkt, dass es in Europa in Summe bereits gelingt, sich bezüglich BIP vom EV absolut zu entkoppeln. Es liegt also auf "kollektiver Ebene" eine absolute Entkopplung vor!
- Welche Länder hier wie genau abschneiden, können wir hier allerdings noch nicht erkennen

<b>Saisonale Trends</b>:
- Wir können bei der Quartalsansicht (anhand der Hoverdata an den lokalen Maxima der Kurven) bereits sehr gut erkennen, dass das BIP in der Ländersumme jedes Jahr in Q4 einen Peak hat, während der EV einen nachgelagerten Peak in Q1 hat.

<b>Zusatzerkenntnisse</b>:
- Bei den Jahresbeträgen erkennen wir bei BIP und EV deutlich den Knick durch die Coronakrise 2020
- Bei den Jahresbeträgen sehen wir außerdem beim BIP einen Knick im Jahr 2009, der eventuell eine Folge der Weltwirtschaftskrise 2008/2009 ist.
- Diesen Knick sehen wir auch beim EV. Dort sehen wir ausserdem bei 2014 und 2017 kleine Knicks. Diese könnten eventuell auf die Krimkrise 2014 und eine weitere Krise am Finanzmarkt 2017 zurückzuführen sein. Da dies jedoch Spekulation ist, bleiben wir bei der reinen Beobachtung, dass es hier Unregelmäßigkeiten gab.

<div style="border: 2px solid black; padding: 10px; background-color: white;">
    <b>Qualitative Haupterkenntnisse</b>
    <ul style="list-style-type: circle;">
            <li>Die sechs Länder Deutschland, Frankreich, Italien, Spanien, Niederlande, Türkei dominieren wirtschaftlich und beim Energieverbrauch.
            </li>
            <li>Wir haben bereits die <b>Absolute Entkopplung (AE) der europäischen Ländersumme</b> sichtbar gemacht.. Außerdem lassen sich Quartalstrends erahnen.
            </li>
        </ul>
</div>


## 1.2 Korrelationsuntersuchung: Zusammenhang BIP und Energieverbrauch über Zeit
Wir wollen nun über die Summe der europäischen Länder hinweg untersuchen, ob es eine Korrelation zwischen BIP und EV in Europa gibt. 

Genauer gesagt: Wir nehmen für jedes Jahr das Wertepaar (BIP, EV) für jedes Land. Damit haben wir pro Jahr eine Stichprobe (im Umfang der Anzahl der Länder), für die wir die Korrelation zwischen BIP und EV als Indikator ausrechnen können. Das machen wir für jedes Jahr, und betrachten, wie sich der Korrelationskoeffizient über die Zeit ändert. Außerdem ziehen wir einen Scatterplot heran, auf dem wir für jedes Jahr die Stichprobe aus BIP und EV visualisieren, und eine Regressionsgerade bilden. 

Wir betrachten dafür zunächst die untenstehende Tabelle, die uns den Korrelationskoeffizienten für jedes Jahr ausgibt. Dabei werden Zellen mit Korr.koeff. größer Null automatisch rot dargestellt.

Anschließend nutzen wir den Scatterplot, der uns über eine Animation die Möglichkeit gibt, die zeitliche Änderung der Regressionsgeraden zwischen BIP und EV nachzuvollziehen.

### 1.2.1 Visualisierung

In [4]:
from dashboards_and_tables.T1_corr_tables.table_builders import correlation_table

correlation_table(df_Y, mode = "Jahre", html_elements_color=html_elements_color).show()

In [5]:
from dashboards_and_tables.D2_correlations.graph_builders import correlations_moving_regression

correlations_moving_regression(df_Y, gdp_color, energy_color).show()


### 1.2.3 Interpretation der verwendeten statistischen Größen
Wir werden im nächsten Teilkapitel die konkreten, qualitativen Ergebnisse zu unseren Daten, die wir aus der Tabelle und dem Plot gewinnen können, zusammenfassen.

Hier wollen wir innehalten und auf die statistischen Werkzeuge eigehen, die wir oben verwendet haben (Korrelationskoeffizient und Regressionsgerade). Da wir diese über die Zeit betrachten (jeweils pro Jahr beide Werkzeuge heranziehen) lohnt es sich, sich hier genau klar zu machen, was Änderungen der beiden Werte über die Zeit bedeuten.

<b>Bedeutung gesamtheitlicher (alle Länder) Korrelationskoeffizient BIP/EV über die Zeit</b>
- Wie wir sehen, ist der Korrelationskoeffizient in jedem Jahr fast 1. Das bedeutet, dass es einen starken positiven linearen Zusammenhang zwischen den Messgrößen BIP und EV gibt.
- Das rechtfertigt zum einen, dass wir der Regressionsgeraden eine hohe Bedeutung zukommen lassen können (die im Plot auch für schwache Korrelationen generiert werden kann, dann aber weniger Aussagekraft hätte)
- Dass der Korrelationskoeffizient über die Jahre leicht sinkt, bedeutet zum anderen aber auch, dass der lineare Zusammenhang zwischen BIP und EV über die Zeit (leicht) schwächer wird. 
    - Das heißt, dass mehr und mehr Länder über die Zeit vom linearen Zusammenhang abweichen - <b>in verschiedene Richtungen</b>! Die <b>Streuung um die Regressionsgerade herum wird über die Zeit größer</b>, also bewegen sich z.B. mehr Länder bei etwa gleichbleibendem BIP beim EV stärker nach oben ODER unten, oder andersherum (allgemeiner: eine der beiden Größen ändert sich gering, die andere ändert sich stärker)
    - Das hat bei genauer Überlegung keine direkte Auswirkung auf die Steigung der Regressionsgeraden (die über die Zeit auch immer gleich bleiben könnte, trotz zunehmender Streuung um die Gerade herum, für die der Korrelationskoeffizient das relevante Maß darstellt). Wir müssen also beide Werkzeuge in Kombination betrachten.
- Der <b>Korrelationskoeffizient</b> zeigt uns hier also die <b>Beständigkeit der linearen Beziehung</b>, aber auch, dass diese leicht zurückgeht! Alleinstehend ist er jedoch nicht als Maß für eine absolute Entkopplung verwendbar, die wir in unserem Kontext suchen.

<b>Bedeutung Steigung der Regressionsgeraden</b>
- Wir haben oben verstanden, dass aufgrund der starken linearen Korrelation über die Zeit (Korrelationskoeffizient) die Regressionsgerade hier eine hohe Aussagekraft hat.
- Das bedeutet, dass wir anhand der Steigung der Regressionsgeraden ablesen können, <b>mit welchem Faktor Änderungen des EV von Änderungen des BIP abhängen</b>.  
- Wie wir sehen, nimmt die Steigung der Regressionsgeraden (bei in jedem Jahr hohem Korrelationskoeffizienten) über die Zeit zu.
    - Dies ist die <b>numerisch-statistische Konsequenz</b> der Tatsache, dass der (über die Länder summierte) europäische EV über die Zeit sinkt, während das BIP weiter wächst! Dies war bereits die Beobachtung im Graphen der summierten Jahresbeträge im vorigen Kapitel.
- Die <b>zunehmende Steigung</b> der Regressionsgeraden ist hier also unser <b>Indikator für eine AE</b> (absolute Entkopplung) zwischen BIP und EV <b>in der europäischen Summe</b>.

<b>Vorüberlegung Energieproduktivität (EP)</b>
- Wir können bereits eine wichte Vorüberlegung zur EP treffen, die sich aus dem Quotienten aus BIP und EV ergibt:
    - Die EP entspricht der <b>Steigung einer Geraden im Scatterplot:</b>
        - Bei gleichem Faktor X vor BIP und EV kürzt sich bei der EP (=BIP/EV) dieser Faktor wieder weg!
        - Damit haben <b>alle Länder</b>, die im Scatterplot <b>auf derselben Geraden</b> liegen (egal welcher) dieselbe EP!
        - Länder auf einer steileren Geraden haben somit eine höhere EP!
        - Liegt ein Land auf einer Geraden weiter rechts/oben, so hat es (mit gleichem Faktor) höhere BIP- und EV-Werte als Ein Land auf derselben Geraden weiter links unten. Das Land ist also "schwergewichtiger" (wirtschaftlich "größer")


Wir können bereits jetzt weitere statistische Merkmale für ein Land mit EV ableiten, die wir im Fall der AE des Landes beobachten werden (äquivalent zur "einfachen" Kategorisierung durch steigendes BIP und sinkenden EV):
 <div style="border: 2px solid black; padding: 10px; background-color: white;">
    <b>Statistische Hauptmerkmale eines Landes bei AE (gleichbedeutend zur "einfachen" Beschreibung über steigendes BIP und sinkenden EV)</b>
    <ul style="list-style-type: circle;">
            <li>Ein Land muss eine steigende EP haben bei gleichbleibend oder sinkendem Energieverbrauch!
            </li>
            <li>Beachte: Die EP kann steigen, während der Energieverbrauch ebenfalls steigt! Das entspricht im Scatterplot einer Bewegung nach oben rechts, und stellt <b>keine AE</b> dar!
            </li>
            <li>Ein Land kann auch bei <b>niedriger EP eine AE</b> haben, aber die <b>EP über die Zeit muss steigen</b>!
            </li>
            <li>Auf Landesebene werden wir später sehen, dass dann für ein Land mit AE der <b>Korrelationskoeffizient zwischen BIP und EV für die Zeitreihe dieses Landes negativ</b> wird (die <b>umgekehrte</b> Schlussfolgerung gilt jedoch <b>im allgemeinen nicht</b>!). <b>Achtung</b>: Dieser Korrelationskoeffizient der Landeszeitreihe wird anders gebildet und hat nichts mit der obigen Tabelle und dem Scatterplot oben zu tun!
        </ul>
</div>

### 1.2.4 Beobachtungen und Haupterkenntnisse

<b>Verteilung und absolute Entkopplung</b>:
- Wir halten einen starken (aber zurückgehenden) linearen Zusammenhang zwischen BIP und EV fest.
- Der leichte Rückgang des Zusammenhangs bedeutet: Es gibt einzelne Länder, die über die Zeit bei "geringen" BIP-Änderungen "hohe" EV-Änderungen aufweisen, oder andersherum.
    - Das zeigt uns an, dass es einige "Vorreiterländer" für die europäische AE gibt, und einige "Gegenläufer", die hinterherhinken, oder sich sogar in die Gegenrichtung bewegen.
    - "Vorreiter" sind im Scatterplot Länder, deren Punkt sich tendenziell <b>nach oben und/oder links bewegt</b> (unabhängig davon, ob der Punkt oberhalb- oder unterhalb der Geraden liegt). Diese Bewegungsrichtung spiegelt eine AE des Landes wieder.
    - "Gegenläufer" sind im Scatterplot Länder, deren Punkt sich tendenziell <b>nach rechts und/oder unten bewegt</b> (unabhängig davon, ob der Punkt oberhalb- oder unterhalb der Geraden liegt).
- Rechte Plothälfte (wirtschaftlich "große" Länder Deutschland, Frankreich, Italien, Spanien, Türkei):
    - Hier erkennen wir über die Hoverdaten den AE-Vorreiter Deutschland und den AE-Gegentreiber Türkei. Spanien, Italien, Frankreich liegen sehr genau auf der Geraden.
- Linke Plothälfte (wirtschaftlich "kleinere" Länder):
    - Hier erkennen wir (auch unter Verwendung des Zooms), dass die Punktwolke sich über die Zeit etwas mehr streut. Es bleibt aber schwierig, hier bei einzelnen Ländern eine Vorreiter- oder Gegentreiberrolle zu erkennen. Das werden wir aber später auf Ebene der Einzelländeruntersuchung gut können.
- In der Gesamtheit überkompensieren die Vorreiter also die Gegenläufer, weshalb gesamteuropäisch die AE gelingt.

 <div style="border: 2px solid black; padding: 10px; background-color: white;">
    <b>Qualitative Haupterkenntnisse</b>
    <ul style="list-style-type: circle;">
            <li>Wir haben nun nochmal auf andere Art (nicht über die Ländersumme sondern) <b>über die Stichprobenstreuung jedes Jahres die Absolute Entkopplung (AE) in der Gesamtheit der europäischen Länder</b> abgebildet.
            </li>
            <li>Damit nimmt in der europäischen Gesamtheit die Energieproduktivität (EP) zu (das wissen wir durch unsere Vorüberlegungen bereits jetzt, ohne es überhaupt visualisiert zu haben!!)
            </li>
            <li>Hierfür gibt es Vorreiterländer und Gegentreiberländer.
            </li>
            <li>Die Vorreiterländer überkompensieren die Gegenläuferländer.
            </li>
            <li>Ein einzelnes Land mit einer AE hat in der Konsequenz bei gleichbleibend/sinkendem EV auch eine steigende EP.
        </ul>
</div>

## 1.3 Europäischer Durchschnitt

Wir betrachten nun die europäischen Durchschnittswerte für BIP, EV und EP.

### 1.3.1 Visualisierung

In [6]:
df = df_Y
from dashboards_and_tables.D3_averages.app import run_averages_app

run_averages_app(df)


### 1.3.2 Beobachtungen und Haupterkenntnisse

<b>Verteilung</b>:
- DFISNT:
    - Wir sehen hier sehr (über die Legende und Hoverdaten) sehr deutlich, dass die wirtschaftsstarken Länder DFISNT bei BIP und EV über die ganze Zeit hinweg über dem Durchschnitt liegen. 
    - Bei der EP hingegen fallen diese Länder auf eher durchschnittliche Werte ab (was logisch ist, da die EP als Quotient gebildet wird). 
    - Die Türkei rutscht sogar stark nach unten. Das überrascht ebenfalls nicht, da wir die Gegenläufereigenschaft vorher schon festgestellt haben.
- Die anderen Länder befinden sich damit mit teilweise sehr geringen BIP- und EV-Verläufen eher oder auch deutlich unter dem Durchschnitt.
    - Polen, Schweden und Norwegen haben allerdings ebenfalls einen überdurchschnittlichen EV bei durchschnittlich bis unterdurchschnittlichem BIP (und damit eine niedrige EP).
    - Bemerkenswert ist, dass Irland, Luxemburg und Dänemark bei der EP mit Abstand sehr gut abschneiden (oberste drei Länder). Wir wissen aber, dass diese steigen muss, während der EV fällt! Wir können uns die einzelnen Verläufe für jedes Land hier schon zusammenklicken (und wären damit eigentlich auch schon bei der Einzelländeranalyse angelangt). Das ist aber hier noch etwas mühsam, daher warten wir, bis wir zur Einzelländeransicht mit einer einfacheren Visualisierung kommen.

<b>Absolute Entkopplung</b>:
- Wir sehen bei den Durchschnittslinien, dass der europäische Durchschnitts für das BIP steigt, während der durchschnittliche EV sinkt. Wir haben hier also die <b>AE des europäischen Durchschnitts (und damit die steigende Durchschnitts-EP)</b> visualisiert.
- Wir können uns hier mit Geduld bereits für jedes Land alle notwendigen Informationen in den Ansichten zusammenklicken, um festzustellen, ob ein Land eine AE hat. Das ist allerdings hier noch mühsam. 

<b>Energieproduktivität</b>:
- Wir sehen, dass die Durchschnittlinie der <b>Energieproduktivität steigt</b>. Dies ist direkte Konsequenz aus der oben festgestellten AE im europäischen Durchschnitt.

<div style="border: 2px solid black; padding: 10px; background-color: white;">
    <b>Qualitative Haupterkenntnisse</b>
    <ul style="list-style-type: circle;">
            <li>Wir haben die <b>europäische AE nun auch auf der Durchschnittsebene</b> sichtbar gemacht.
            </li>
            <li>Damit nimmt auch im Durchschnitt die Energieproduktivität (EP) zu.
            </li>
            <li>Wir können bereits hier jedes einzelne Land vollständig auf AE untersuchen. Das nehmen wir aber anhand nutzerfreundlicherer Visualisierungen im nächsten Kapitel vor.
        </ul>
</div>



# 2. Einzelanalyse auf Landesebene

Nachdem wir uns nun die europäischen Ländersummen und Durchschnitte angesehen haben, wollen wir auf die Detailebene jedes Land einzeln auf Absolute Entkopplung (AE) untersuchen.

Hierfür werden wir für jedes Land einzeln die BIP-, EV-, und EP-Werte über die Zeit betrachten. Wir beginnen mit der Untersuchung der Korreltion zwischen BIP und EV für jedes Land.

## 2.1 Korrelationen BIP und EV über Zeit (Untersuchung absolute Entkopplung)
Wir haben weiter oben bereits festgestellt:
- Auf Landesebene muss für ein Land mit AE der <b>Korrelationskoeffizient zwischen BIP und EV für die Zeitreihe dieses Landes negativ</b> sein (die <b>umgekehrte</b> Schlussfolgerung gilt jedoch <b>im allgemeinen nicht</b>!). 
- Daraus können wir <b>in der Regel</b> direkt schlussfolgern, dass das BIP steigt, während der EV fällt. Der Fall, dass das BIP fällt, während der EV steigt (was auch zu einem negativen Korrelationskoeffizienten führt!) ist aufgrund der politischen Prämisse eines anzustrebenden Wirtschaftswachstums sehr unrealistisch (wir werden aber sehen, dass Griechenland seit 2008 BIP-Verluste hat!).

Wir bestimmen daher nun für jedes einzelne Land den Korrelationskoeffizienten zwischen BIP und EV, und lassen uns das tabellarisch ausgeben.
- Beachte: Hier nehmen wir also <b>nicht wie vorher pro Jahr eine Stichprobe</b>, deren Umfang (Anzahl Messpaare BIP/EV) sich aus der <b>Anzahl der Länder</b> ergibt (wie im vorigen Kapitel), sondern wir nehmen <b>jetzt pro Land eine Stichprobe</b>, deren Umfang (Anzahl Messpaare BIP/EV) sich aus der <b>Anzahl der Jahre</b> ergibt!

Dabei werden in der Tabelle Zellen mit negativen Korr.koeff. kleiner als -0.2 grün dargestellt (das indiziert die erwünschte AE), Zellen mit betragsmäßig kleinem Korr.koeff. (zwischen -0.2 und +0.2) grau, Zellen mit Korr.koeff. größer als +0.2 rot (Gegenläufer zur AE). Wir machen das einmal auf Jahresebene (kleinerer Stichprobenumfang pro Land) und einmal auf Quartalsebene (größerer Stichprobenumfang pro Land). 

### 2.1.1 Korrelationsberechnung

### Jahresebene

In [7]:
unique_countries = sorted(df_Y['Land'].unique())
first_half_of_countries = unique_countries[:len(unique_countries) // 2]
second_half_of_countries = unique_countries[len(unique_countries) // 2:]
df_Y_first_half = df_Y[df_Y['Land'].isin(first_half_of_countries)].sort_values(by='Land')
df_Y_second_half = df_Y[df_Y['Land'].isin(second_half_of_countries)].sort_values(by='Land')

In [8]:
correlation_table(df_Y_first_half, mode = "Land", html_elements_color=html_elements_color).show()
correlation_table(df_Y_second_half, mode = "Land", html_elements_color=html_elements_color).show()

### Quartalsebene

In [9]:
df_Q_first_half = df_Q[df_Q['Land'].isin(first_half_of_countries)].sort_values(by='Land')
df_Q_second_half = df_Q[df_Q['Land'].isin(second_half_of_countries)].sort_values(by='Land')
correlation_table(df_Q_first_half, mode = "Land", html_elements_color=html_elements_color).show()
correlation_table(df_Q_second_half, mode = "Land", html_elements_color=html_elements_color).show()

### 2.1.2 Auswertung Korrelationsberechnung
- Vergleich Jahresebene und Quartalsebene: 
    - Für jedes Land wird der <b>Korr.koeff. im absoluten Betrag (vorzeichenfrei) kleiner</b>, wenn wir <b>von der Jahresebene in die Quartalsebene</b> gehen. 
    - Das bedeutet, dass die Korrelation auf Quartalsebene "verrauscht". Das ergibt Sinn, da die <b>"Wellenbewegung" von BIP und EV in ihren Peaks nicht zusammenfällt</b> (was wir bereits bei der Betrachtung im allerersten Kapitel gesehen haben).
- Ablesen Länder mit AE:
    - Wir können <b>direkt die Länder mit grünen Zellen als sichere AE-Länder</b> ablesen.
    - Länder mit <b>roten</b> Zellen haben <b>keine AE im Sinne der Korrelation</b>.
        - Hier müssen wir sehr genau aufpassen: 
            - Der Korr.koeff. zeigt uns, dass wir <b>keine AE als statistischen (langfristigen!) Trend in der Zeitreihe</b> haben!!
            - Es <b>kann aber sehr wohl (!) Länder geben</b>, die eine <b>kurzfristige AE</b> durch rapide EV-Reduktion in der jungen Vergangenheit erreichen. Diese "Ausreißer in der Stichprobe" des Landes müssen <b>nicht zwangsläufig einen negativen Korr.koeff.</b> (was bei uns der grüne Bereich ist) haben, da der Ausreißer den Korr.koeff. ggf. nicht stark genug verändert! <b>Tatsächlich werden wir solche Länder finden!</b> 
    - Länder mit <b>grauen</b> Zellen (Wortspiel unbeabsichtigt) sollten wir in jedem Fall separat betrachten.

## 2.2 Visualisierung
Wir ziehen nun ein umfangreiches Dashboard heran, auf dem wir jedes Land genauestens bezüglich BIP, EV, EP, und Korrelation zwischen BIP und EV betrachten können.

In [10]:
from dashboards_and_tables.D4_country_ana.app import country_analysis_run_app

country_analysis_run_app(df_Y, df_Q, df)

## 2.3 Beobachtungen und Haupterkenntnisse

<b>Absolute Entkopplung</b>:
- Wir haben nun spezifisch für jedes Land festgestellt, ob eine absolute Entkopplung vorliegt, oder nicht. Liegt eine AE vor, so gilt einer der folgenden Fälle:
    - Der Korrelationskoeffizient der BIP-EV-Zeitreihe des Landes negativ, und die EP-Zeitreihe des Landes steigt über die Zeit (langfristige AE).
    - Der Korrelationskoeffizient der BIP-EV-Zeitreihe des Landes ist positiv, aber es gab irgendwo in der Zeitreihe (in unseren gefundenen Ländern Litauen und Estland am Ende der Zeitreihe) eine starke Reduktion des EV bei gleichzeitigem BIP-Anstieg (kurzfristige AE).
    - Der Korrelationskoeffizient hat einen geringen absoluten Betrag (bei Griechenland, Norwegen, Rumänien und der Slovakei), und wir prüfen die AE visuell im Dashboard.

Die Ergebnisse können wir der folgenden Tabelle entnehmen.


<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        table {
            border-collapse: collapse;
            width: 100%;
        }
        th, td {
            border: 2px solid black;
            text-align: left;
            padding: 8px;
        }
        th {
            font-weight: bold;
        }
        ul {
            list-style-type: circle;
            margin: 0;
            padding-left: 20px; /* Bullet-Indent */
        }
    </style>
</head>
<body>
    <table>
        <tr>
            <th>Langfristige (ab 2008) absolute Entkopplung (Korr.Koeff. > +0.2)</th>
            <th>Keine langfristige absolute Entkopplung (Korr.koeff < -0.2)</th>
        </tr>
        <tr>
            <td>
                <ul>
                    <li>Austria</li>
                    <li>Belgium</li>
                    <li>Bulgaria</li>
                    <li>Croatia</li>
                    <li>Czechia</li>
                    <li>Denmark</li>
                    <li>Finland</li>
                    <li>France</li>
                    <li>Germany</li>
                    <li>Italy</li>
                    <li>Latvia</li>
                    <li>Luxembourgh</li>
                    <li>Netherlands</li>
                    <li>Slovenia</li>
                    <li>Spain</li>
                    <li>Sweden</li>
                </ul>
            </td>
            <td>
                <ul>
                    <li>Kein Trend zur AE</li>
                            <ul style="list-style-type: circle;">
                                <li>Cyprus</li>
                                <li>Ireland</li>
                                <li>Malta</li>
                                <li>Norway</li>
                                <li>Portugal</li>
                                <li>Türkiye</li>
                            </ul>
                    <li>Kurzfristige AE (Trendentwicklung seit 2021)</li>
                            <ul style="list-style-type: circle;">
                                <li>Estonia</li>
                                <li>Greece</li>
                                <li>Hungary</li>
                                <li>Lithuania</li>
                                <li>Poland</li>
                                <li>Romania</li>
                                <li>Slovakia</li>
                            </ul>
                </ul>
            </td>
        </tr>
    </table>
</body>
</html>

Abschließend:

<div style="border: 2px solid black; padding: 10px; background-color: white;">
    <b>Qualitative Haupterkenntnisse</b>
    <ul style="list-style-type: circle;">
            <li>AE: Alle Länder wurden <b>genauestens</b> bezüglich AE untersucht. Damit wurde auch die EP in Bezug auf ihren zeitlichen Verlauf mituntersucht. 
            </li>
            <li>Wir haben damit unsere Untersuchung auf AE vollständig vorgenommen und abgeschlossen.</li>
        </ul>
</div>

# 3. Ländervergleich (Rankings)
Wir wollen nun die Länder detaillierter auf Rankings untersuchen: Wer steht bei BIP, EV, EP an welcher Stelle im Vergleich zum Rest. 

<b>Beachte</b>: Wir betrachten hier nur <b>Jahresscheiben, keine zeitlichen Verläufe</b>. Wir wollen hier also keine weiteren Erkenntnisse bezüglich AE generieren. Hier geht es uns eher wieder um einen genaueren Überblick bezüglich Wirtschaftsstärke (BIP) und EV/EP pro Jahresscheibe.

Wir betrachten dafür folgendes Dashboard mit verschiedenen Sortieroptionen und der Möglichkeit, ein einzelnes Land durch Klick auf einen der Balken des Landes optisch zu markieren.

### 3.1 Visualisierung

In [11]:
from dashboards_and_tables.D5_country_comp.app import country_comparison_run_app

country_comparison_run_app(df_Y)

### 3.2 Beobachtungen und Haupterkenntnisse

<b>Verteilung</b>:
- Bei Sortierung nach BIP: 
    - Hier sehen wir nochmal sehr deutlich, dass die Länder DFISNT bei BIP und EV hoch, bei der EP eher durchschnittlich abschneiden. Die Niederlande schneiden hier noch am besten ab. Die Türkei reißt beim EV massiv nach oben aus und ist bei der EP entsprechend sehr weit unten (beides bereits bekannt).

<b>Absolute Entkopplung</b>:
- Bei Betrachtung der wirtschaftsschwächeren Länder (Option "ohne 6 größte BIP-Erzeuger") sehen wir, dass der Zusammenhang "hohes BIP = hoher EV" stärker zu schwanken beginnt. Bei DFISNT dagegen herrscht noch ein sehr klarer Zusammenhang. Beides hatten wir zu Beginn bereits im Scatterplot grob sehen können, hier wird das nochmal deutlicher. 
- Das ist ein Anzeichen dafür, dass es den "Wachstumsländer" Europas bereits bei geringer wirtschaftlicher Größe gelingt, vom linearen Zusammenhang "hohes BIP = hoher EV" wegzukommen, und einen Wachstumspfad bei AE zu verfolgen. Bei einer erfolreichen AE würde der Zusammenhang bei Wirtschaftswachtum dann womöglich durchbrochen.  

<b>Energieproduktivität</b>:
- Bei Sortierung nach BIP: 
    - Wir sehen nochmal gut, dass bei den Ländern im "BIP-Mittelfeld" oder darunter Irland, Dänemark und Luxemburg nach oben ausreißen.
- Bei Sortierung nach EV sehen wir auch warum: Die EP-Ausreißer Irland, Dänemark, Luxemburg haben sehr niedrige EV-Werte (logisch).
    - Beachte: Wir wissen aus den vorigen Kapiteln, dass Irland allerdings keine AE vorweist! Luxemburg und Dänemark allerdings schon. 
- Bei Sortierung nach BIP und Anzeige ohne DFISNT (Option "ohne 6 größte BIP-Erzeuger") sehen wir außerdem nochmal klarer, dass Polen, Schweden und Norwegen bei den "Niedrig-BIP-Ländern" nochmal beim EV nach oben ausschlagen (und entsprechend bei der EP sehr schwach abschneiden). Das kommt hier deutlicher zum Vorschein als in vorigen Plots.

<b>Qualitative Haupterkenntnisse</b>
 <div style="border: 2px solid black; padding: 10px; background-color: white;">
    <ul style="list-style-type: circle;">
            <li>DFISNT dominieren bei BIP und EV stark. Dort herrscht ein gut sichtbarer Zusammenhang zwischen hohem BIP und hohem EV.
            </li>
            <li>Die Länder im wirtschaftlichen Mittel- und Schlussbereich weisen einen weniger klaren Zusammenhang zwischen hohem BIP und hohem EV auf. Das haben wir bereits zu Beginn im Scatterplot erahnen können.
            </li>
            <li>Die Wirtschaftsgröße (BIP) hat keinen erkennbaren Zusammenhang zur EP. Wirtschaftsstärke bedeutet also nicht gleich hohe Energieproduktivität
            </li>
            <li>Ebenso hat der Energieverbrauch (EV) hat keinen erkennbaren Zusammenhang zur EP.
            </li>
        </ul>
</div>

# 4. Geografische Untersuchung

Wir wollen nun eine geografische Visualisierung vornehmen. Wir betrachten zum Einen exemplarisch die Verteilung von BIP und EV im Jahr 2023. Zum Anderen betrachten wir die prozentualen Änderungen je Land bei BIP und EV in Bezug auf das Referenzjahr 2008 (das am weitesten zurückliegende Jahr in unserem durch die Daten gegebenen Referenzzeitraum). Dann werden wir für 2023 noch die EP betrachten.

## 4.1 BIP und Energieverbrauch 2023 und absolute Entkopplung seit 2008

### 4.1.1 Feature Engineering der benötigten Spalten
Wir wollen in der Folge Choroplethenplots verwenden, die Länder auf der europäischen Karte gemäß einer Wert-Farbe-Zuordnung einfärben. 

Wir haben jedoch bereits gesehen, dass DFISNT bei BIP und EV dominieren, während sich viele der übrigen Länder mit geringen Werten im unteren Wertebereich "zusammendrängeln". Wir wollen vermeiden, dass aufgrund dieser Verteilung die Farbskalen unserer folgenden Plots im niedrigen Bereich keine angemessene farbliche Differenzierung hervorbringen. Daher führen wir Hilfsspalten ein, in denen wir die BIP- und EV-Werte logarithmieren. Zudem setzen wir die Farbskalen in den Plots von Hand.

Wir führen also die Spalten zu prozentualen Änderung ein sowie zusätzliche Spalten für logarithmierte Werte.

In [12]:
# BIP und Energieverbrauch logarithmisch transformieren (für Farbskala)
df_Y['BIP_logarithmiert'] = np.log1p(df_Y['BIP (Mio €)']).round(2)
df_Y['Energieverbrauch_logarithmiert'] = np.log1p(df_Y['Energieverbrauch (GWh)']).round(2)
# relative Änderung im Vergleich zu 2008 berechnen (Achtung: NICHT logarithmiert!)
for column in ['BIP (Mio €)', 'Energieverbrauch (GWh)']:
    df_Y = df_Y.merge(df_Y[df_Y['Jahr'] == 2008][['Land', column]], on='Land', suffixes=('', '_2008'))
    df_Y[f'{column}_Diff zu 2008 (%)'] = ((df_Y[column] / df_Y[f'{column}_2008']) * 100 - 100).round(1)

### 4.1.2 Visualisierung

In [13]:
from dashboards_and_tables.D6_geo_analysis.apps import run_app1

run_app1(df_Y)

### 4.1.3 Beobachtungen und Haupterkenntnisse
<b>Verteilung</b>:
- Wir können hier sehr gut erkennen: 
    - Die Spitzenwerte in 2023 bei BIP und EV ballen sich in West- und Zentraleuropa sowie der Türkei. Ostblock, baltische Länder und Skandinavien liegen im mittleren bis unteren Bereich.
    - Beim BIP-Wachstum schneiden West- und Zentraleuropa allerdings eher mittel ab. Dagegen holt der Ostblock sichtbar auf (Spitzenwert bei Bulgarien), ebenso Irland. Italien stagniert, Griechenland hingegen hat seit 2008 an Wirtschaftskraft verloren.

<b>Absolute Entkopplung</b>:
- Wir sehen sehr deutlich, dass seit 2008 in Zentral- und Westeuropa eine vergleichsweise starke AE stattfindet (Ausnahme: Portugal). 
- Den östlichen Ländern gelingt (trotz ihres vergleichsweise sogar <b>höheren Wirtschaftswachstums</b>) bis auf Polen <b>ebenfalls eine AE</b> (wenn auch prozentual schwächer). Auch in Skandinavien ist eine AE erkennbar.
- Zypern, Irland, Malta, Portugal, und die Türkei steigern jedoch aktuell parallel zu ihrem BIP-Wachstum auch ihren EV.

<b>Qualitative Haupterkenntnisse</b>
 <div style="border: 2px solid black; padding: 10px; background-color: white;">
    <ul style="list-style-type: circle;">
            <li>Zentral- und Westeuropa:
                <ul style="list-style-type: circle;">
                    <li>Hier konzentrieren sich 2023 sehr (!) <b>hohe</b> BIP- und <b>hohe</b> EV-Werte (beachte die logarithmierte Farbskala für 2023!).</li>
                    <li>Gleichzeitig seit 2008 ein prozentual eher <b>geringes</b> Wirtschaftswachstum bei gleichzeitig <b>starker</b> AE.</li>
                </ul>
            </li>
            <li>Skandinavien:
                <ul style="list-style-type: circle;">
                    <li>In 2023 <b>mittlere</b> BIP- und vergleichweise <b>hohe</b> EV-Werte (beachte die logarithmierte Farbskala für 2023!).</li>
                    <li>Gleichzeitig seit 2008 ein prozentual eher <b>geringes</b> Wirtschaftswachstum bei gleichzeitig <b>starker</b> AE.</li>
                </ul>
            </li>
            <li>Ostblock und baltische Länder:
                <ul style="list-style-type: circle;">
                    <li>In 2023 <b>niedrig bis mittlere</b> BIP- und EV-Werte (beachte die logarithmierte Farbskala für 2023!).</li>
                    <li>Gleichzeitig seit 2008 ein prozentual eher <b>mittel bis hohes</b> Wirtschaftswachstum bei gleichzeitig <b>starker</b> AE.</li>
                </ul>
            </li>
            <li>Auffällige Länder:
                <ul style="list-style-type: circle;">
                    <li>"Krisenland" Griechenland: Einbruch bei BIP-Wachstum.</li>
                    <li>"Kleine Umweltsünder" Zypern, Malta: mittlere EV-Steigerung bei auf niedrige EV-Werte 2023</li>
                    <li>"Große Klimasünder" Portugal, Türkei, Irland: Steigerung auf hohe Werte. Polen auch, aber Trend zur AE (siehe vorige Kapitel).</li>
                </ul>
            </li>
        </ul>
</div>

## 4.2 Energieproduktivität 2023

### 4.2.1 Visualisierung

In [14]:
from dashboards_and_tables.D6_geo_analysis.apps import run_app2
run_app2(df_Y)

### 4.2.2 Beobachtungen und Haupterkenntnisse
<b>Energieproduktivität</b>:
- Wir sehen, dass im Jahr 2023 Zentral- und Westeuropa eine tendenziell hohe EP hat (Deutschland ganz vorne).
- Der Ostblock (der hohes BIP-Wachstum verzeichnet, gleichzeitig aber auch den EV reduziert und damit eine AE umsetzt) hat dennoch eher niedrige EP-Werte.
    - Das ist, wie wir nun des Öfteren gesehen haben, durchaus möglich. Bei einer AE (BIP steigt, EV fällt) kann die EP niedrig sein, die AE ist nur durch <b>Steigerung der EP</b> gekennzeichnet!
- Irland brilliert auf den ersten Blick abermals, wir wissen aber, dass es <b>keine AE</b> vorweist!
-  Skandinavien schneidet eher schlecht ab, aber wir wissen, dass zumindest Schweden und Finnland einen Trend zur AE haben!
- Bei der Türkei wissen wir, dass eine Gegenläuferbewegung zur AE stattfindet, was hier auch zur niedrigen EP passt.

<b>Qualitative Haupterkenntnisse</b>
 <div style="border: 2px solid black; padding: 10px; background-color: white;">
    <ul style="list-style-type: circle;">
            <li>Zentral- und Westeuropa:
                <ul style="list-style-type: circle;">
                    <li>Eher hohe EP-Werte. Wir wissen bereits, dass hier AE vorliegt, EP also weiter steigt.</li>
                </ul>
            </li>
            <li>Skandinavien:
                <ul style="list-style-type: circle;">
                    <li>Geringe EP-Werte (wir wissen aber aus vorigen Kapiteln: steigender Trend).</li>
                </ul>
            </li>
            <li>Ostblock und baltische Länder:
                <ul style="list-style-type: circle;">
                    <li>Geringe EP-Werte ((wir wissen aber aus vorigen Kapiteln: steigender Trend)).</li>
                </ul>
            </li>
            <li>Auffällige Länder:
                <ul style="list-style-type: circle;">
                    <li>Irland und Luxemburg hohe Werte. Bei Luxemburg AE, bei Irland nicht (im Gegenteil!)</li>
                </ul>
            </li>
        </ul>
</div>

# 5. Saisonale Analyse

Wir wollen nun die Quartalstrends beleuchten. Wir haben bereits bei der kumulierten BEtrachtung zu Beginn der EDA einen deutlichen und einheitlichen Trend festgestellt: In der europäischen Summe peakt das BIP in Q4, der Energieverbrauch in Q1.

Wir wollen uns das nochmals genauer anschauen, indem wir dieses "pulsierende" Verhalten geografisch visualisieren. Dafür normieren wir die Quartalswerte anteilig an das jeweilige Jahr. Später werden wir noch die Häufigkeit betrachten, mit der ein Quartal je Land und in Europa den Maximalwert des Jahres annimmt.

## 5.1 Visualisierung "pulsierende" Jahresmaxima EP
Wir wollen im Folgenden die jährlichen Peaks der Energieproduktivität visualisieren. Bei BIP und EV haben wir ja schon im ersten Kapitel gut erkennen können, dass Q4 (beim BIP) und Q1 (beim EV) Jahrespeaks haben. Wir schauen uns nun noch die EP an, der, wie wir sehen werden, ebenfalls solche Peaks hat.

### 5.1.1 Feature Engineering benötigter Spalten

Wir definieren zunächst den "Erreichungsgrad der Jahresmaximalproduktivität" der EP als Anteil eines Quartalswertes am Jahresmaximum des Jahres, in dem er liegt. Dafür definieren wir uns folgende Spalte, die wir dann in der Visualisierung verwenden.

In [15]:
df_Q["Erreichungsgrad_Jahresmaximalproduktivität"] = (
    (df_Q["Energieproduktivität (Mio€/GWh)"] - df_Q.groupby(["Land", "Jahr"])["Energieproduktivität (Mio€/GWh)"].transform("min")) /
    (df_Q.groupby(["Land", "Jahr"])["Energieproduktivität (Mio€/GWh)"].transform("max") - df_Q.groupby(["Land", "Jahr"])["Energieproduktivität (Mio€/GWh)"].transform("min"))
)

### 5.1.2 Visualisierung

In [16]:
from dashboards_and_tables.D6_geo_analysis.apps import run_app3

run_app3(df_Q)

OK


### 5.1.3 Beobachtungen und Haupterkenntnisse
<b>Saisonale Trends</b>:
- Wir sehen sehr deutlich:
    - In Q1 ist die EP fast immer und fast überall im Vergleich zum Restjahr am niedrigsten.
    - In Q2 und Q3 ist die EP (zumindest nach optischem Eindruck) am häufigsten maximal. In Q4 eher mittelmäßig oft.

<b>Qualitative Haupterkenntnisse</b>
 <div style="border: 2px solid black; padding: 10px; background-color: white;">
    <ul style="list-style-type: circle;">
            <li>Q1 ist absolutes Flaute-Quartal für die EP. In Q2 und Q3 ist sie tendenziell am höchsten.</li>
        </ul>
</div>

## 5.2 Numerische Häufigkeitsanalyse der Jahresmaxima 

Nach der Visualisierung der jährlichen EP-Maxima wollen wir nun noch in Zahlenform alle Größen (BIP, EV, EP) auf Häufigkeit der Jahresmaxima untersuchen. Wir benötigen dafür wieder neue Spalten (in einem Hilfs-Dataframe).

### 5.2.1 Feature Engineering benötigter Spalten
Wir führen nun Zusatzspalten in einer pivotisierten Variante unserer Daten ein.

In [17]:
# Pivot-Tabelle mit mehreren Spalten für 'Energieverbrauch (GWh)', 'BIP' und 'Produktivität'
pivot_df = df_Q.pivot_table(index=['Land', 'Jahr'], columns='Quartal', values=['Energieverbrauch (GWh)', 'BIP (Mio €)', 'Energieproduktivität (Mio€/GWh)'])

# Maximalwerte für jede der Spalten finden und als neue Spalten hinzufügen
pivot_df['Max_Quartal_Energieverbrauch'] = pivot_df['Energieverbrauch (GWh)'].idxmax(axis=1)
pivot_df['Max_Quartal_BIP'] = pivot_df['BIP (Mio €)'].idxmax(axis=1)
pivot_df['Max_Quartal_Energieproduktivität'] = pivot_df['Energieproduktivität (Mio€/GWh)'].idxmax(axis=1)

pivot_df_reset = pivot_df.reset_index()

# Die relevanten Spalten auswählen und in das gewünschte Format bringen
formatted_df = pivot_df_reset[['Land', 'Jahr', 'Max_Quartal_Energieverbrauch', 'Max_Quartal_BIP', 'Max_Quartal_Energieproduktivität']]

### 5.2.2 Visualisierung

Wir visualisieren uns die erhaltenen Werte in einem Dashboard, das uns Histogramme zeigt. Dargestellt wird die Häufigkeit von Quartalen, mit der sie Maximumswert ihres Jahres für das ausgewählte Land (bzw. alle Länder) waren.

In [18]:
from dashboards_and_tables.D6_geo_analysis.apps import run_app4

run_app4(df, formatted_df)

### 5.2.3 Beobachtungen und Haupterkenntnisse
<b>Saisonale Trends</b>:
- In Europa (über alle Länder hinweg) wird das BIP am häufigsten in Q4 maximal (wirtschaftlicher Endjahres-Boom, Weihnachtsgeschäft, Auftrags- und Bilanzabschlüsse, ...).
- Dagegen ist der Energieverbrauch Q1 am häufigsten maximal (Heizperiode).
- Die EP ist in Q3 am häufigsten maximal.

<b>Qualitative Haupterkenntnisse</b>
 <div style="border: 2px solid black; padding: 10px; background-color: white;">
    <ul style="list-style-type: circle;">
            <li>Q4 ist in Europa absolutes Boom-Quartal für das BIP.</li>
            <li>In Q1 dagegen ist in Europa EV maximal.</li>
        </ul>
</div>

# 6. Abschluss und Zusammenfassung

Wir haben nun eine ausführliche Reise quer durch verschiedene Aggregationsebenen der europäischen Länder vorgenommen. Dabei haben wir statistische Werkzeuge wie den Korrelationskoeffizienten für verschiedene Stichproben und Regressionsgeraden als Hilfsmittel verwendet. In allen Kapiteln hatten wir dabei unsere Kernfragen der absoluten Entkopplung und Energieproduktivität dabei.

Folgende Teiluntersuchungen haben wir auf unserer Reise vorgenommen:
- Wir haben
    - eine kumulierte Betrachtung der europäischen Länder vorgenommen,
    - europäische Durchschnittswerte unter die Lupe genommen,
    - und den Zusammenhang zwischen BIP und Energieverbrauch auf gesamteuropäischer Ebene sowie auf Ebene einzelner Länder untersucht.
- Dabei haben wir
    - auf gesamteuropäischer Ebene die absolute Entkopplung der Gesamtheit Europas betrachtet,
    - sowie für jedes Land einzeln untersucht, ob eine absolute Entkopplung beobachtbar ist.
- Anschließend haben wir die Länder in Bezug auf Wirtschaftsstärke verglichen, und überprüft, ob ein Zusammenhang zwischen Wirtschaftskraft und Energieproduktivität besteht.
- Wir haben geografisch visualisiert, ob sich bestimmte Ländergruppen anders Verhalten, und Länder mit Sonderverhalten hervorgehoben.
- Abschließend haben wir unsere Analysegrößen auf saisonale Trends untersucht.

Wir schließen nun unsere EDA ab, indem wir gesammelt nochmals die Hauptergebnisse zu unseren Leitfragen (die wir kursiv aufführen) zusammenfassen:
<div style="border: 2px solid black; padding: 10px; background-color: white;">
    <ul style="list-style-type: circle;">
            <li><b><i>Verteilung</i></b>:
                <ul style="list-style-type: circle;">
                    <li><i>Gibt es Länder, die bezüglich einer (oder mehrerer) der Größen BIP, Energieverbrauch ("EV"), Energieproduktivität ("EP") dominieren?</i></li>
                        <ul style="list-style-type: circle;">
                            <li>Die sechs Länder Deutschland, Frankreich, Italien, Spanien, Niederlande, Türkei (="DFISNT") dominieren wirtschaftlich (BIP) und beim EV stark</li>
                                <ul style="list-style-type: circle;">
                                    <li>Sie weisen allerdings ein eher niedriges Wirtschaftswachstum (BIP-Wachstum) auf</li>
                                </ul>
                            <li>Die restlichen Länder bilden ein Mittelfeld und einen breiten unteren Bereich mit niedrigen Werten.</li>
                                <ul style="list-style-type: circle;">
                                    <li>Sie liegen beim Wirtschaftswachstum allerdings vorne. Insbesondere Ostblock und baltische Länder holen bei der Wirtschaftsstärke (BIP-Wachstum) auf. Dabei gelingt es diesen Ländern allerdings, ihren EV nicht zu steigern.</li>
                                </ul>
                        </ul>
                    <li><i>Wie verhält es sich bei diesen Ländern in den jeweils anderen Größen?</i></li>
                        <ul style="list-style-type: circle;">
                            <li>DFISNT haben jeweils sehr hohe BIP- und EV-werte, hier ist ein klarer Zusammenhang erkennbar. In dieser Ländergruppe sticht zudem die Türkei durch einen besonders hohen EV (gemessen am BIP) hervor. Bei der EP ist es gemischt. Deutschland und die Niederlande haben noch vergleichsweise eine hohe EP. Die anderen DFISNT-Länder liegen eher im Mittelfeld, die Türkei jedoch hat eine sehr niedrige EP. Hohe BIP- und EV-Werte stehen also in keinem erkennbaren Zusammenhang zu hohen EP-Werten.</li>
                        </ul>
                </ul>
            </li>
            <li><i><b>Absolute Entkopplung</b></i>:
                <ul style="list-style-type: circle;">
                    <li><i>Gibt es einen Zusammenhang zwischen hohem BIP und hohem Energieverbrauch?</i></li>
                        <ul style="list-style-type: circle;">
                            <li>Bei den DFISNT-Ländern ist dieser klar erkennbar. Hier findet jedoch seit 2008 auch eine deutliche absolute Entkopplung ("AE") statt. </li>
                        </ul>
                    <ul style="list-style-type: circle;">
                        <li>Im Mittelfeld der BIP-Werte (weniger wirtschaftsstarke Länder) ist erkennbar, dass der Zusammenhang zwischen hohem BIP und hohem EV weniger ausgeprägt ist. Dort ist zudem erkennbar, dass der Zusammenhang über die Zeit schwächer wird (was wiederum an der AE vieler Vorreiterländer in diesem Wirtschaftsgrößenbereich liegt).</li>
                    </ul>
                    <li><i>Gibt es einen Trend davon weg?</i></li>
                        <ul style="list-style-type: circle;">    
                            <li>Ja! Übergreifend ist erkennbar, dass die (länderübergreifende) Korrelation zwischen BIP und EV jedes Jahr sinkt.</li>   
                        </ul>
                        <ul style="list-style-type: circle;">
                            <li>In der europäischen Ländersumme und ebenso bei den europäischen Durchschnittswerten liegt bereits eine klare AE vor. Diese wird insbesondere durch einen großen Teil an Vorreiterländern erreicht, die eine geringere Zahl an Ländern mit gegenläufigem Trend überkompensieren.</li>
                        </ul>    
                    <ul style="list-style-type: circle;">    
                            <li>Konkret weisen bereits 16 der 29 betrachteten Länder eine langfristige AE auf. Bei weiteren 7 Ländern ist seit 2021 der Trend dorthin (kurzfristige AE) erkennbar. Lediglich 6 Länder lassen noch keinen Trend zu einer AE erkennen.</li>   
                        </ul>
                    <li><i>Welchen Ländern gelingt es, sich zu entkoppeln (BIP steigt, Energieverbrauch sinkt)?</i></li>
                        <ul style="list-style-type: circle;">
                            <li>Die konkreten Länderlisten können Kapitel 2.3 entnommen werden!</li>
                        </ul>
                </ul>
            </li>
            <li> <i><b>Energieproduktivität</b></i>:
                <ul style="list-style-type: circle;">
                    <li><i>Gibt es bei der Energieproduktivität einen Zusammenhang zur Wirtschaftsstärke (BIP) oder der absoluten Entkopplung (AE) eines Landes?</i>
                        <ul style="list-style-type: circle;">
                        <li>Zusammenhang EP und BIP: 
                            <ul style="list-style-type: circle;">
                                <li>Die EP als Quotient aus BIP und EV hängt über die Korrelation zwischen EV und BIP (bei festem Jahr über die Länder hinweg) vom BIP ab. Zwischen BIP und EV haben wir (bei festem Jahr über die Länder hinweg) eine lineare Abhängigkeit zwischen BIP und EV festgestellt. Diese spiegelt sich (bei festem Jahr über die Länder hinweg) auch in der EP wieder. Wenn wir die Länder der Wirtschaftsstärke nach durchgehen und dabei die EV-Werte vergleichen (Ranking), spiegeln die EV-Ausschläge sich in den EP-Ausschlägen (reziprok) wieder.</li>
                            </ul>
                        <li>Zusammenhang EP und AE:</li>
                        <ul style="list-style-type: circle;">
                            <li>Zwischen EP und AE gibt es außerdem einen direkten zeitlichen Zusammenhang! Die AE ist (definitionsgemäß) ein zeitliches Phänomen. Bei einem Land mit AE steigt die EP über die Zeit. Der Umkehrschluss (steigende EP führt zu AE) stimmt im Allgemeinen nicht (Irland als Gegenbeispiel).</li>
                        </ul>
                    </ul>
                    </li>
                </ul>
            </li>
            <li><i><b>Saisonalität</b></i>:
                <ul style="list-style-type: circle;">
                    <li><i>Sind saisonale Trends bei BIP, Energieverbrauch und Energieproduktivität erkennbar?</i>
                        <ul style="list-style-type: circle;">
                            <li>Ganz klar: In Q4 boomt das BIP in Europa (Jahresabschlüsse, Weihnachtsverkäufe, Rechnungsabschlüsse, Bilanzen vollmachen, ...), in Q1 geht der EV in die Höhe (Heizen...)! Die EP ist tendenziell in Q3 maximal.</li>
                        </ul>
                    </li>
                </ul>
            </li>
            <li><i><b>Zusatzerkenntnisse</b></i>:
                <ul style="list-style-type: circle;">
                    <li><i>Welche weiteren interessanten Beobachtungen können wir machen?</i>
                        <ul style="list-style-type: circle;">
                            <li>Wir konnten deutlich den Coronaknick in BIP und EV sehen. Auch andere Zeitpunkte weisen Knicke in BIP und EV auf, die mit europaweiten politischen Einschnitten zusammenfallen (Finanzkrise 2008; Krimkrise 2014; nachgelagerte Finanzkrise 2017)</li>
                        </ul>
                    </li>
                </ul>
            </li>
        </ul>
</div>


<b>Nach dieser ausführlichen Zusammenfassung haben wir unsere EDA beendet. Wir haben also mit hauptsächlich visuellen Mitteln (und zusätzlichen Analysen der Korrelationskoeffizienten über die beiden Dimensionen Zeit (Jahre) und Geografie (Länder) hinweg) enorm viele Erkenntnisse aus den Daten generieren können. Wir bedanken uns bei unseren Lesern und schließen.</b>